In [2]:
import os
import argparse
from bertopic import BERTopic
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# TODO  check babyplot for 3D visualisation

/Users/kiliansprenkamp/miniconda3/envs/refugeeAnalysis/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# testing

In [ ]:
def read_data(input_file):
    with open(input_file) as file:
        lines = file.readlines()
        text_to_analyse_list = [line.rstrip() for line in lines]
    return text_to_analyse_list

In [ ]:
data = read_data("../data/BERTopicInput.csv")

In [ ]:
model = BERTopic(verbose=True, language="multilingual", nr_topics=30) # vectorizer_model=vectorizer_model)#, nr_topics=20)
topics, probs = model.fit_transform(data)

In [ ]:
import sys
import itertools
# setting path
sys.path.append('../src/BERTopic/')
from runBERTopic import BERTopicAnalysis

nr_topics_list = ["auto"]
min_topic_size_list = [10,25,50,100,200,400,800,1000,1500]
hbscan_min_cluster_size_list = [25,50,100,200]


for i in itertools.product(nr_topics_list,min_topic_size_list,hbscan_min_cluster_size_list):
    if os.path.exists(f"../models/BERTopic50Char{i[0].capitalize()}Classes{i[1]}MinTopicSize{i[2]}HbscanMinClusterSize/BERTopicmodel"):
        print('wuhu', i)
        continue
    BERTopic_Analysis = BERTopicAnalysis(input_file="../data/BERTopicInput50CharMin.csv",
                                         k_cluster=i[0],
                                         min_topic_size=i[1],
                                         hbscan_min_cluster_size=i[2]
                                     )
    print(i)    
    BERTopic_Analysis.run_all()


In [ ]:
with open("../data/BERTopicInput50CharMin.csv") as file:
    lines = file.readlines()
    text_to_analyse_list = [line.rstrip() for line in lines]

In [ ]:
df = pd.read_csv("../models/BERTopic_50Char10Classes/pred_on_whole_data_translated_Sentiment.csv")

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
fp = open("test.txt")
data = fp.read()
tokenizer.tokenize(data)

In [ ]:
def inference(x):
    if len(x) >= 500:
        for i in tokenize.sent_tokenize(x):
            print(i)

In [ ]:
df_value_counts = df[df.chat.isin(['https://t.me/zh_helps_ukraine', 'https://t.me/zh_helps_UArefugee',
       'https://t.me/zurich_hb_help', 'https://t.me/zh_housing',
       'https://t.me/Zurich_UA', 'https://t.me/zh_helps_UArefugees', 'https://t.me/Zh_helps_UA_mums', 'https://t.me/job_sw_ukrainians',
       'https://t.me/zh_back_ukraine', 'https://t.me/zh_helps_logistics',])].value_counts(['cluster', 'week']).reset_index()
df_value_counts.columns = ['cluster', 'week', 'count']
mydict = {'no_class':-1,
          'medical': 0, 
          'teaching': 1,
          'banking': 2,
          'transport_UKR_CH': 3,
          'pets': 4,
          'immigration': 5,
          'transport_train_EU': 6,
          'website_links': 7,
          'volunteering': 8,
          'transport_train_CH': 9}
df_value_counts['cluster_names'] = df_value_counts['cluster'].apply(lambda x: list(mydict.keys())[list(mydict.values()).index(x)])

In [ ]:
df_actors = pd.DataFrame(columns=["cluster_name", "message_sender", "nr_post"])
for i in df.cluster_name.unique():
    df[df['cluster_name']==i].value_counts('messageSender').iloc[:10]
    df_actors_i = df[df['cluster_name']==i].value_counts('messageSender').iloc[:10].reset_index()
    df_actors_i["cluster_name"] = i
    df_actors_i = df_actors_i[["cluster_name", "messageSender", 0]]
    df_actors_i.columns = ["cluster_name", "message_sender", "nr_post"]
    df_actors = pd.concat([df_actors, df_actors_i], axis=0)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("../models/BERTopic100CharMin2500CharMax_10ClassesRedo/df.csv")
df['uniqueIdentifier'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageID"]), axis=1)
df['uniqueIdentifierReply'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageReplyID"])[:-2], axis=1)
df = df[~df.chat.isin(["https://t.me/campax_ukraine_help_switzerland", "https://t.me/helppetsfromukraine"])] 
mydict = {'no_class':-1,
          'medical': 0, 
          'legal_issues': 1,
          'travel': 2,
          'immigration': 3,
          'childcare': 4,
          'education': 5,
          'veterinarian': 6,
          'banking': 7,
          'passport_information': 8,
          'weblinks': 9}
df['cluster_names'] = df['cluster'].apply(lambda x: list(mydict.keys())[list(mydict.values()).index(x)])
df['week']=df['messageDatetime'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])
df_value_counts = df.groupby(['cluster','week']).count().reset_index()

In [ ]:
import plotly.express as px
# for i in df_value_counts.cluster.unique()[1:]:
fig = px.line(df_value_counts[df_value_counts.cluster_names != "no_class"].sort_values(['week']), x="week", y="count", color='cluster_names', title='Cluster over time')
fig.show()
# fig.write_html("../models/BERTopic_50Char10Classes/AllGroups.html")

In [ ]:
import plotly.express as px
# for i in df_value_counts.cluster.unique()[1:]:
fig = px.line(df_value_counts[df_value_counts.cluster != -1].sort_values(['week']), x="week", y="count", color='cluster_names', title='Cluster over time')
fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
for i in df_value_counts.cluster.unique()[1:]:
    plt.plot(df_value_counts[df_value_counts.cluster == i].sort_values(['week'])['week'], df_value_counts[df_value_counts.cluster == i].sort_values(['week'])['count'], linestyle = 'dotted', label=list(mydict.keys())[list(mydict.values()).index(i)])
plt.legend(loc="upper left")
plt.show()

# Sentiment

In [45]:
df = pd.read_csv("../models/BERTopic100CharMin2500CharMax_25Classes_dropDuplicates/df.csv")
print(df.shape)
df = df[df.cluster!=-1]
print(df.shape)
# df = df[df.messageTextEnglish.apply(type) == str]
# print(df.shape)
# df = df[df['messageTextEnglish'].notnull()]
# print(df.shape)
# df[df["messageText"].str.len()<512]
# df = df[df['messageTextEnglish'].str.len() <= 513]
# df.reset_index(inplace=True)

(404276, 10)
(65154, 10)


In [48]:
df.cluster.value_counts()

0     9266
1     5036
17    4774
3     4519
2     3709
5     3349
4     3298
7     3236
12    3123
10    2335
16    2137
6     2055
9     1934
13    1902
11    1877
14    1600
8     1585
19    1464
20    1455
22    1451
18    1387
15    1324
24     987
23     682
21     669
Name: cluster, dtype: int64

In [56]:
df[df.cluster==19].sample(500).messageText.to_list()

['Друзі, а є тут перукарі чи барбери? Або хтось може щось порадити, де можна підстригтися якісно та за адекватний прайс, бажано звичайно наших (хочеться підтримати власного виробника😄) Потрібна звичайна чоловіча стрижка, без бороди. Кантон Цюрих в принципі весь підходить.',
 'Все добрый день. Занимаюсь коллористикой и наращиваем волос, цены приемлимые, не Швейцарские😄 прикрепляю ниже маленькую часть моих работ, кто хочет изменить имидж или же убрать неудачно сделанную работу мастера обращайтесь. Я живу в Uzwil, могу приехать на дом если недалеко 😉',
 'Делала стрижку каре по адресу Grand Rue 6, за 25 fr. Майя +41 79 711 64 80 ватсап',
 'Елена, здравствуйте! Если есть возможность, то с большим удовольствием бы взяла машинку для стрижки в пользование А то сыну 4 года быстро волосы отростают, уже ходи стричься (дорогое удовольствие конечно)',
 'Добрый день! Девочки, подскажите где можно купить краску для волос недорого?',
 'Нужен женский парикмахер? напишите в личку пожалуйста',
 'Парикмах

In [16]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("T'estimo!")


[{'label': 'positive', 'score': 0.6600586771965027}]

In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
PYTORCH_ENABLE_MPS_FALLBACK=1
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)#.to("mps")
model.to("mps")
text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt').to("mps")
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


NotImplementedError: The operator 'aten::cumsum.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [5]:
text

'Good night 😊'

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")#.to("mps")

In [5]:
labels = ['Negative', 'Neutral', 'Positive']
def analyse_sentiment(x):
    if len(x)>512:
        x_split_list = []
        n = 400
        for i in range(0, len(x), n):
            x_split_list.append(x[i:i+n])
        scores_split_list = []    
        for x_split in x_split_list:
            encoded_tweet = tokenizer(x_split, return_tensors='pt')#.to("mps")
            output = model(**encoded_tweet)
            scores_split = output[0][0].detach().numpy()
            scores_split = softmax(scores_split)
            scores_split_list.append(scores_split)
        return np.mean(np.array(scores_split_list), axis=0)
        
    else:
        encoded_tweet = tokenizer(x, return_tensors='pt')#.to("mps")
        output = model(**encoded_tweet)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        return scores
        
def scores_to_sentiment(x):
    max_value = max(x.tolist())
    max_index = x.tolist().index(max_value)
    return labels[max_index] 
    

In [25]:
sum(df[df.cluster!=-1].messageText.str.len())
10218946
 1000000

10218946

In [18]:
x = "👋Вітаємо на каналі КОРИСНО (HELPFUL) Швейцарського Червоного Хреста. Ми — група волонтерів, яка допоможе вам знайти відповіді на ваші запитання. Ми намагаємось відповідати впродовж 24 годин. Відповіді на запитання надаються з понеділка по п’ятницю. Перш ніж поставити своє запитання, просимо вас спочатку переглянути веб-сайт КОРИСНО (HELPFUL): helpful.redcross.ch На цьому каналі можна використовувати українську, російську, англійську та державні мови Швейцарії. Ми відповідатимемо здебільшого державними мовами Швейцарії та пропонуватимемо перекласти наші відповіді. Для цього скористайтеся функцією «перекладу», яку пропонує Telegram. Указівки та коментарі наведено нижче. ⚠️Оскільки це публічний канал, ваші коментарі бачитимуть всі бажаючі. Якщо у вас є запитання, що передбачає розкриття особистої інформації, просимо вас додати в коментарі значок ✉️. Тоді ми надішлемо вам приватне повідомлення в зашифрованому чаті. 📍Відповіді на деякі питання були надані в приватних повідомленнях. Просимо вас писати під відповідними секціями в основному каналі. ____ 👋Добро пожаловать на канал HELPFUL Швейцарского Красного Креста. Мы - команда волонтеров и готовы помочь вам найти ответы на ваши вопросы. Мы стараемся давать ответы в течение 24 часов. Мы будем отвечать на ваши вопросы с понедельника по пятницу. Прежде чем задать свой вопрос, просим вас сначала заглянуть на сайт HELPFUL: helpful.redcross.ch. На этом канале используются украинский, русский, английский или швейцарские государственные языки. Мы будем отвечать преимущественно на языках, используемых в Швейцарии, и просим вас переводить наши ответы. Для этого воспользуйтесь опцией ""перевести"", имеющейся в Telegram. Смотрите инструкции в комментариях ниже. ⚠️Поскольку этот канал является публичным, ваши комментарии сможет увидеть каждый. Если у вас есть вопрос, подразумевающий предоставления каких-то личных данных, просим вас отправить значок ✉️ в комментарии. После этого мы отправим вам личное сообщение в защищенном чате. ___ 👋 Welcome to the channel HELPFUL by the Swiss Red Cross. We are a team of volunteers and will help you find answers to your questions. We aim to respond within 24 hours. The questions will be answered from Monday to Friday. Before asking your question, we ask you to please first check the HELPFUL website: helpful.redcross.ch The languages used in this channel are Ukrainian, Russian, English or the Swiss national languages. We will mainly answer using the Swiss national languages and ask you to translate our answers. To do this please use the option “translate” provided by Telegram. See instructions in the comments below. ⚠️ As this channel is public, your comments will be seen by everyone. If you have a question that requires sharing some personal data, we ask you to send the icon ✉️ in the comment. We will then send you a private message in a secured chat. 📍Some answers are provided in private messages. We also please ask you to write under the appropriate sections in the main channel."
x1 = "Dear group members, We are pleased to announce that the **second part of the HELPFUL website** about staying in Switzerland is already online 🎉 Please check out the published information on popular topics such as: work, education, finding accommodation and language learning here ➡️ https://helpful.redcross.ch/uk/perebuvannya We hope that you will find a lot of useful information for yourself and if you have any questions, we are ready to answer them. Have a nice day! Sincerely, HELPFUL Team"
x_split1 = analyse_sentiment(df.messageText[0])

In [19]:
scores_to_sentiment(x_split1)

'Neutral'

In [103]:
x_split1

array([0.12221589, 0.8199387 , 0.05784534], dtype=float32)

In [92]:
np.mean(np.array(x_split), axis=0)

array([0.08510411, 0.78612053, 0.1287754 ], dtype=float32)

In [72]:
x_split["input_ids"].shap

torch.Size([1, 565])

In [38]:
x = "👋Вітаємо на каналі КОРИСНО (HELPFUL) Швейцарського Червоного Хреста. Ми — група волонтерів, яка допоможе вам знайти відповіді на ваші запитання. Ми намагаємось відповідати впродовж 24 годин. Відповіді на запитання надаються з понеділка по п’ятницю. Перш ніж поставити своє запитання, просимо вас спочатку переглянути веб-сайт КОРИСНО (HELPFUL): helpful.redcross.ch На цьому каналі можна використовувати українську, російську, англійську та державні мови Швейцарії. Ми відповідатимемо здебільшого державними мовами Швейцарії та пропонуватимемо перекласти наші відповіді. Для цього скористайтеся функцією «перекладу», яку пропонує Telegram. Указівки та коментарі наведено нижче. ⚠️Оскільки це публічний канал, ваші коментарі бачитимуть всі бажаючі. Якщо у вас є запитання, що передбачає розкриття особистої інформації, просимо вас додати в коментарі значок ✉️. Тоді ми надішлемо вам приватне повідомлення в зашифрованому чаті. 📍Відповіді на деякі питання були надані в приватних повідомленнях. Просимо вас писати під відповідними секціями в основному каналі. ____ 👋Добро пожаловать на канал HELPFUL Швейцарского Красного Креста. Мы - команда волонтеров и готовы помочь вам найти ответы на ваши вопросы. Мы стараемся давать ответы в течение 24 часов. Мы будем отвечать на ваши вопросы с понедельника по пятницу. Прежде чем задать свой вопрос, просим вас сначала заглянуть на сайт HELPFUL: helpful.redcross.ch. На этом канале используются украинский, русский, английский или швейцарские государственные языки. Мы будем отвечать преимущественно на языках, используемых в Швейцарии, и просим вас переводить наши ответы. Для этого воспользуйтесь опцией ""перевести"", имеющейся в Telegram. Смотрите инструкции в комментариях ниже. ⚠️Поскольку этот канал является публичным, ваши комментарии сможет увидеть каждый. Если у вас есть вопрос, подразумевающий предоставления каких-то личных данных, просим вас отправить значок ✉️ в комментарии. После этого мы отправим вам личное сообщение в защищенном чате. ___ 👋 Welcome to the channel HELPFUL by the Swiss Red Cross. We are a team of volunteers and will help you find answers to your questions. We aim to respond within 24 hours. The questions will be answered from Monday to Friday. Before asking your question, we ask you to please first check the HELPFUL website: helpful.redcross.ch The languages used in this channel are Ukrainian, Russian, English or the Swiss national languages. We will mainly answer using the Swiss national languages and ask you to translate our answers. To do this please use the option “translate” provided by Telegram. See instructions in the comments below. ⚠️ As this channel is public, your comments will be seen by everyone. If you have a question that requires sharing some personal data, we ask you to send the icon ✉️ in the comment. We will then send you a private message in a secured chat. 📍Some answers are provided in private messages. We also please ask you to write under the appropriate sections in the main channel."

my_list = []

n = 512

for i in range(0, len(x), n):
    my_list.append(x[i:i+n])

print(my_list)  # 👉️ ['ab', 'cd', 'ef', 'gh']


['👋Вітаємо на каналі КОРИСНО (HELPFUL) Швейцарського Червоного Хреста. Ми — група волонтерів, яка допоможе вам знайти відповіді на ваші запитання. Ми намагаємось відповідати впродовж 24 годин. Відповіді на запитання надаються з понеділка по п’ятницю. Перш ніж поставити своє запитання, просимо вас спочатку переглянути веб-сайт КОРИСНО (HELPFUL): helpful.redcross.ch На цьому каналі можна використовувати українську, російську, англійську та державні мови Швейцарії. Ми відповідатимемо здебільшого державними мовам', 'и Швейцарії та пропонуватимемо перекласти наші відповіді. Для цього скористайтеся функцією «перекладу», яку пропонує Telegram. Указівки та коментарі наведено нижче. ⚠️Оскільки це публічний канал, ваші коментарі бачитимуть всі бажаючі. Якщо у вас є запитання, що передбачає розкриття особистої інформації, просимо вас додати в коментарі значок ✉️. Тоді ми надішлемо вам приватне повідомлення в зашифрованому чаті. 📍Відповіді на деякі питання були надані в приватних повідомленнях. Пр

In [109]:
len(my_list[5])

437

In [111]:
'models/BERTopic100CharMin2500CharMax_20/df.csv'.split('.')[0]

'models/BERTopic100CharMin2500CharMax_20/df'

In [35]:
x = "👋Вітаємо на каналі КОРИСНО (HELPFUL) Швейцарського Червоного Хреста. Ми — група волонтерів, яка допоможе вам знайти відповіді на ваші запитання. Ми намагаємось відповідати впродовж 24 годин. Відповіді на запитання надаються з понеділка по п’ятницю. Перш ніж поставити своє запитання, просимо вас спочатку переглянути веб-сайт КОРИСНО (HELPFUL): helpful.redcross.ch На цьому каналі можна використовувати українську, російську, англійську та державні мови Швейцарії. Ми відповідатимемо здебільшого державними мовами Швейцарії та пропонуватимемо перекласти наші відповіді. Для цього скористайтеся функцією «перекладу», яку пропонує Telegram. Указівки та коментарі наведено нижче. ⚠️Оскільки це публічний канал, ваші коментарі бачитимуть всі бажаючі. Якщо у вас є запитання, що передбачає розкриття особистої інформації, просимо вас додати в коментарі значок ✉️. Тоді ми надішлемо вам приватне повідомлення в зашифрованому чаті. 📍Відповіді на деякі питання були надані в приватних повідомленнях. Просимо вас писати під відповідними секціями в основному каналі. ____ 👋Добро пожаловать на канал HELPFUL Швейцарского Красного Креста. Мы - команда волонтеров и готовы помочь вам найти ответы на ваши вопросы. Мы стараемся давать ответы в течение 24 часов. Мы будем отвечать на ваши вопросы с понедельника по пятницу. Прежде чем задать свой вопрос, просим вас сначала заглянуть на сайт HELPFUL: helpful.redcross.ch. На этом канале используются украинский, русский, английский или швейцарские государственные языки. Мы будем отвечать преимущественно на языках, используемых в Швейцарии, и просим вас переводить наши ответы. Для этого воспользуйтесь опцией ""перевести"", имеющейся в Telegram. Смотрите инструкции в комментариях ниже. ⚠️Поскольку этот канал является публичным, ваши комментарии сможет увидеть каждый. Если у вас есть вопрос, подразумевающий предоставления каких-то личных данных, просим вас отправить значок ✉️ в комментарии. После этого мы отправим вам личное сообщение в защищенном чате. ___ 👋 Welcome to the channel HELPFUL by the Swiss Red Cross. We are a team of volunteers and will help you find answers to your questions. We aim to respond within 24 hours. The questions will be answered from Monday to Friday. Before asking your question, we ask you to please first check the HELPFUL website: helpful.redcross.ch The languages used in this channel are Ukrainian, Russian, English or the Swiss national languages. We will mainly answer using the Swiss national languages and ask you to translate our answers. To do this please use the option “translate” provided by Telegram. See instructions in the comments below. ⚠️ As this channel is public, your comments will be seen by everyone. If you have a question that requires sharing some personal data, we ask you to send the icon ✉️ in the comment. We will then send you a private message in a secured chat. 📍Some answers are provided in private messages. We also please ask you to write under the appropriate sections in the main channel."


In [34]:
from tqdm import tqdm
tqdm.pandas()

df['sentiment'] = df['messageText'].progress_apply(lambda x: analyse_sentiment(x))
df['sentiment_name'] = df['sentiment'].progress_apply(lambda x: scores_to_sentiment(x))

100%|██████████| 414616/414616 [00:00<00:00, 1092111.94it/s]


In [15]:
df_test= df_test.sort_values("sentiment_name")

In [23]:
df = pd.read_csv("..models/BERTopic100CharMin2500CharMax_25Classes_dropDuplicates/df_prep_sentimet.csv/df_sentimet.csv")

In [18]:
# df = df[df.cluster==0]

In [24]:
df['week']=df['messageDatetime'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])


In [29]:
df_value_counts = df.value_counts(["week", "sentiment_name_twitter_xlm_roberta_base_sentiment"]).reset_index()

In [33]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df_value_counts, x="week", y=0, color="sentiment_name_twitter_xlm_roberta_base_sentiment", title="Sentiment within cluster")
fig.show()

In [ ]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df_test, x="cluster_name", y="percentage", color="sentiment_name", title="Sentiment within cluster")
fig.show()

In [ ]:
for i in df.cluster.unique():
    print(i)
    fig1, ax1 = plt.subplots()
    ax1.pie(df[df.cluster==i].value_counts('sentiment_name').values, labels=df[df.cluster==i].value_counts('sentiment_name').index, autopct='%1.1f%%',
        shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    title = list(mydict.keys())[list(mydict.values()).index(i)]
    fig1.suptitle(title, fontsize=12)

    

In [ ]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()

In [ ]:
df = pd.read_csv('/Users/kiliansprenkamp/Desktop/OneDrive - Universität Zürich UZH/migrantZurich/unsupervised-migrant-telegram-analysis/models/BERTopic_50Char10Classes/pred_on_whole_data_translated_sentiment_network.csv')

In [ ]:
df.messageText.str.len().nlargest(3000)

# Translation

In [37]:
# transformer for slavic to english

from transformers import MarianTokenizer, MarianMTModel
import torch

# src = "uk"  # source language
# trg = "en"  # target language

model_name = f"Helsinki-NLP/opus-mt-tc-big-zle-en"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)
# tokenizer.to("mps")

# model.to(torch.device("mps"))
sample_text =  df.messageText[0]
batch = tokenizer([sample_text], return_tensors="pt")

generated_ids = model.generate(**batch)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'Dear members of the group, We are glad to announce that **the second part of the HELPFUL website** about staying in Switzerland is already online  Please view the published information on popular topics such as: work, education, housing search and language study here  https://helpful.redcross.ch/uk/perebuvannya Hope you find yourself useful!'

In [41]:
df.shape

(62153, 10)

In [43]:
df.drop_duplicates(subset=['messageText', 'messageSender', 'chat'], keep='first',inplace=True)


In [44]:
df.shape

(59159, 10)

In [40]:
df['messageTextEnglish'] = df['messageText'].progress_apply(lambda x: tokenizer.batch_decode(model.generate(**tokenizer([x], return_tensors="pt")), skip_special_tokens=True)[0])


  0%|          | 32/62153 [05:46<186:43:27, 10.82s/it]


KeyboardInterrupt: 

In [35]:
from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-zle-en", device="mps")
print(pipe("Скільки мені слід купити пива?"))

NotImplementedError: The operator 'aten::remainder.Tensor_out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [32]:
df["messageTextEnglish"] = df["messageText"].progress_apply(lambda x: pipe(x, max_length=2500))

  0%|          | 15/414616 [00:25<196:40:47,  1.71s/it]


KeyboardInterrupt: 

# potential publication

In [6]:
df = pd.read_csv("../models//BERTopic100CharMin2500CharMax_10ClassesNewestData/df.csv")
df['uniqueIdentifier'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageID"]), axis=1)
df['uniqueIdentifierReply'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageReplyID"])[:-2], axis=1)
# df = df[~df.chat.isin(["https://t.me/campax_ukraine_help_switzerland", "https://t.me/helppetsfromukraine"])] 
class_name_dict = {'no_class':-1,
          'Medical': 0, 
          'Complaints': 1,
          'Asylum/Volunteering': 2,
          'Banking': 3,
          'Train Travel': 4,
          'S Status': 5,
          'Accomodation': 6,
          'Veterinarian': 7,
          'Passport Information': 8,
          'Transport UKR CH': 9}
df['cluster_names'] = df['cluster'].apply(lambda x: list(class_name_dict.keys())[list(class_name_dict.values()).index(x)])
df['week']=df['messageDatetime'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])
messages_per_week_dict = dict(df.value_counts("week"))
df_value_counts = df.value_counts(["cluster_names", "week"]).reset_index()
df_value_counts.columns = ["cluster_names", "week", "occurence_count"]
df_value_counts['occurence_count_norm'] = df_value_counts.apply(lambda x: x.occurence_count/list(messages_per_week_dict.values())[list(messages_per_week_dict.keys()).index(x.week)], axis=1)

In [8]:
import plotly.express as px
fig = px.line(df_value_counts[df_value_counts.cluster_names != "no_class"].sort_values(['week']), x="week", y="occurence_count", color='cluster_names', title='Cluster over time')
fig.show()

In [78]:
positive = df[df.uniqueIdentifier=="https://t.me/refugeesinSwitzerland_165886"].messageReactions.values[0]

In [86]:
np.isnan(df.messageReactions[0])

True

In [85]:
if df.messageReactions[0]==float('nan'):
    print('test')

In [41]:
import re

txt = "The rain in Spain"
x = re.search("^The.*Spain$", txt)

In [94]:
count_reaction(positive)

7

In [97]:
import re

def count_reaction(reaction):
    if isinstance(reaction, str):
        m = re.findall('count=(.+?),', reaction)
        m = list(map(int, m))
        return sum(m)
    else:
        return float(0)

In [98]:
df["reaction_count"] = df.apply(lambda x: count_reaction(x.messageReactions), axis=1)

In [99]:
df

,chat,messageSender,messageID,messageReplyID,messageDatetime,messageViews,messageForwards,messageReactions,messageText,cluster,...,prob_5,prob_6,prob_7,prob_8,prob_9,uniqueIdentifier,uniqueIdentifierReply,cluster_names,week,reaction_count
0,https://t.me/campax_ukraine_help_switzerland,-1.001507e+12,1465,NaN,2022-11-09 11:54:20+00:00,349.0,0.0,NaN,Запрошуємо вас на вебінар: ЕНІОСТИЛЬ. Особиста...,-1,...,1.806893e-04,2.828152e-04,8.368974e-06,4.737701e-04,7.745345e-04,https://t.me/campax_ukraine_help_switzerland_1465,https://t.me/campax_ukraine_help_switzerland_n,no_class,45,0.0
1,https://t.me/campax_ukraine_help_switzerland,-1.001507e+12,1464,NaN,2022-11-08 17:11:10+00:00,600.0,3.0,NaN,"9 листопада, в День української писемності та ...",-1,...,4.456777e-73,2.239044e-73,4.911532e-137,2.256454e-73,8.786752e-72,https://t.me/campax_ukraine_help_switzerland_1464,https://t.me/campax_ukraine_help_switzerland_n,no_class,45,0.0
2,https://t.me/campax_ukraine_help_switzerland,-1.001507e+12,1463,NaN,2022-11-08 16:32:02+00:00,557.0,3.0,NaN,Запрошуємо вас у цьому році на три останніх зу...,-1,...,2.655554e-23,4.205600e-23,3.461776e-24,7.345640e-23,1.251607e-22,https://t.me/campax_ukraine_help_switzerland_1463,https://t.me/campax_ukraine_help_switzerland_n,no_class,45,0.0
3,https://t.me/campax_ukraine_help_switzerland,-1.001507e+12,1462,NaN,2022-11-08 13:26:26+00:00,561.0,0.0,NaN,За півгодини починаємо: Campax запрошує Тебе п...,-1,...,2.784245e-53,4.426106e-53,1.945072e-73,7.376132e-53,1.569924e-18,https://t.me/campax_ukraine_help_switzerland_1462,https://t.me/campax_ukraine_help_switzerland_n,no_class,45,0.0
4,https://t.me/campax_ukraine_help_switzerland,-1.001507e+12,1461,NaN,2022-11-08 13:25:25+00:00,538.0,8.0,NaN,BCUE сумісно з PwC PricewaterhouseCoopers пров...,-1,...,1.295806e-135,5.288591e-71,7.583213e-137,3.823073e-135,7.425516e-57,https://t.me/campax_ukraine_help_switzerland_1461,https://t.me/campax_ukraine_help_switzerland_n,no_class,45,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287958,https://t.me/SwissUA,1.264982e+09,307,NaN,2022-02-26 18:54:00+00:00,NaN,NaN,MessageReactions(results=[ReactionCount(reacti...,У нас ще є 6 днів - ми справимось ❤️,5,...,4.259132e-170,5.314623e-170,8.549347e-175,7.095355e-170,4.591511e-152,https://t.me/SwissUA_307,https://t.me/SwissUA_n,S Status,8,1.0
287959,https://t.me/SwissUA,3.368129e+08,306,NaN,2022-02-26 18:41:02+00:00,NaN,NaN,MessageReactions(results=[ReactionCount(reacti...,"если выдает ошибку при подтверждении, подождит...",-1,...,1.113500e-02,1.234151e-02,1.659530e-04,2.000865e-02,2.937823e-02,https://t.me/SwissUA_306,https://t.me/SwissUA_n,no_class,8,1.0
287960,https://t.me/SwissUA,4.852700e+08,305,301.0,2022-02-26 18:39:36+00:00,NaN,NaN,NaN,"Если петиция не открывается, значит очень мног...",-1,...,3.001051e-129,2.930032e-121,1.000877e-130,8.587486e-129,2.994915e-115,https://t.me/SwissUA_305,https://t.me/SwissUA_301,no_class,8,0.0
287961,https://t.me/SwissUA,3.368129e+08,304,301.0,2022-02-26 18:09:06+00:00,NaN,NaN,MessageReactions(results=[ReactionCount(reacti...,спасибо,-1,...,1.534590e-118,1.942554e-112,1.320957e-121,4.284275e-110,9.667967e-92,https://t.me/SwissUA_304,https://t.me/SwissUA_301,no_class,8,1.0


In [23]:
list(df[(df.cluster_names=="Accomodation")].nlargest(10, "messageForwards").messageText)

['https://www.comparis.ch/ 12 Tips як арендувати житло від Comparis: https://www.comparis.ch/immobilien/wohnungssuche/wohnungssuche-tipps-tricks',
 'Сайт пошуку аренди житла: https://www.homegate.ch/de',
 'Помічниця по господарству з проживанням+допомога з дитиною по необхідності ( 4 роки, спокійний, 4 рази в тиждень відвідує дитячий садок). Надається окрема кімната з санвузлом, харчування, допомога з оформленням статусу S. Або ж приходящу помічницю декілька раз на тиждень. Локація - Adliswil 8134. Деталі в ПП будь-ласка https://www.facebook.com/olya.olga.167',
 'Помічниця по господарству з проживанням+допомога з дитиною по необхідності ( 4 роки, спокійний, 4 рази в тиждень відвідує дитячий садок). Надається окрема кімната з санвузлом, харчування, допомога з оформленням статусу S. Або ж приходящу помічницю декілька раз на тиждень. Локація - Adliswil 8134. Деталі в ПП будь-ласка https://www.facebook.com/olya.olga.167',
 '**Ищете квартиру? Хотите отдельное жильё? Как снять его в разумные

In [17]:
df[df.messageForwards.nlargest(10)]

KeyError: "None of [Float64Index([27267.0, 17493.0, 13924.0, 13039.0, 11668.0, 11448.0, 11071.0,\n              10563.0,  9478.0,  9384.0],\n             dtype='float64')] are in the [columns]"

## model after merge topic

In [ ]:
model=BERTopic.load("../models/BERTopic100CharMin2500CharMax_10Classes/merge_topics/BERTopicmodel")

In [ ]:
topics, probabilities = model.fit_transform(docs[:10000])

In [ ]:
model.get_representative_docs().keys()

In [ ]:
writer = pd.ExcelWriter(f"../models/BERTopic100CharMin2500CharMax_10Classes/merge_topics/representative_docs.xlsx", engine='xlsxwriter')
for i in model.get_representative_docs().keys():
    print(i)
    df = pd.DataFrame(model.get_representative_docs()[i], columns=['message'])
    df.to_excel(writer, sheet_name=model.get_topic_info()[model.get_topic_info()['Topic']==i]['Name'].values[0][:31])
writer.save()
model.get_topic_info().to_csv(f"../models/BERTopic100CharMin2500CharMax_10Classes/merge_topics/topic_info.csv")

## model20

In [ ]:
model2=BERTopic.load("../models/BERTopic100CharMin2500CharMax_20Classes/BERTopicmodel")
with open('../data/BERTopicInput100CharMin2500CharMax.csv') as file:
    lines = file.readlines()
    docs = [line.rstrip() for line in lines]

In [ ]:
topics2, probabilities2 = model2.fit_transform(docs)

In [ ]:
import pickle
with open("../models/BERTopic100CharMin2500CharMax_20Classes/probabilities.pkl",'wb') as f:
    pickle.dump(probabilities2, f)
with open("../models/BERTopic100CharMin2500CharMax_20Classes/topics.pkl",'wb') as f:
    pickle.dump(topics2, f)

In [ ]:
with open("../models/BERTopic100CharMin2500CharMax_20Classes/topics.pkl",'rb') as f:
    x = pickle.load(f)


# Multimodal 

In [3]:
import os
import glob
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from sentence_transformers import SentenceTransformer, util

# Flickr 8k images
img_folder = 'photos/'
caps_folder = 'captions/'
if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0:
    os.makedirs(img_folder, exist_ok=True)

    if not os.path.exists('Flickr8k_Dataset.zip'):   #Download dataset if does not exist
        util.http_get('https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip', 'Flickr8k_Dataset.zip')
        util.http_get('https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip', 'Flickr8k_text.zip')

    for folder, file in [(img_folder, 'Flickr8k_Dataset.zip'), (caps_folder, 'Flickr8k_text.zip')]:
        with zipfile.ZipFile(file, 'r') as zf:
            for member in tqdm(zf.infolist(), desc='Extracting'):
                zf.extract(member, folder)
images = list(glob.glob('photos/Flicker8k_Dataset/*.jpg'))

# Prepare dataframe
captions = pd.read_csv("captions/Flickr8k.lemma.token.txt",sep='\t',names=["img_id","img_caption"])
captions.img_id = captions.apply(lambda row: "photos/Flicker8k_Dataset/" + row.img_id.split(".jpg")[0] + ".jpg", 1)
captions = captions.groupby(["img_id"])["img_caption"].apply(','.join).reset_index()
captions = pd.merge(captions, pd.Series(images, name="img_id"), on="img_id")

# Extract images together with their documents/captions
images = captions.img_id.to_list()
docs = captions.img_caption.to_list()


  8%|▊         | 93.8M/1.12G [00:05<01:22, 12.3MB/s]

KeyboardInterrupt: 

In [ ]:
print("wuhu")
model = SentenceTransformer('clip-ViT-B-32')
print("wuhu")
# Prepare images
batch_size = 32
print("wuhu")
nr_iterations = int(np.ceil(len(images) / batch_size))
print("wuhu")
# Embed images per batch
embeddings = []
for i in tqdm(range(nr_iterations)):
    start_index = i * batch_size
    end_index = (i * batch_size) + batch_size

    images_to_embed = [Image.open(filepath) for filepath in images[start_index:end_index]]
    img_emb = model.encode(images_to_embed, show_progress_bar=False)
    embeddings.extend(img_emb.tolist())

    # Close images
    for image in images_to_embed:
        image.close()
embeddings = np.array(embeddings)

In [ ]:
from bertopic import BERTopic
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs, embeddings[:123])
captions["Topic"] = topics